In [1]:
# Imports
from functools import partial
from importlib import reload
import unittest

import qcodes as qc
from qcodes import Instrument
qc.config['core']['legacy_mp'] = True
qc.loops.USE_MP = True

import silq
from silq.pulses import PulseSequence, DCPulse, TriggerPulse, SinePulse
from silq.instrument_interfaces import get_instrument_interface, Channel
from silq.meta_instruments.chip import Chip
from silq.meta_instruments.layout import Layout, Connection, SingleConnection
from silq.meta_instruments.mock_instruments import MockArbStudio, MockPulseBlaster, MockATS, MockAcquisitionController

qc.show_subprocess_widget()

<IPython.core.display.Javascript object>

In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
class TestPulseSequence(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()

    def test_pulse_equality(self):
        pulse1 = DCPulse(name='dc', amplitude=1.5, duration=10, t_start=0)
        self.assertTrue(pulse1)
        pulse2 = DCPulse(name='dc', amplitude=1.5, duration=10, t_start=0)
        self.assertEqual(pulse1, pulse2)
        pulse3 = DCPulse(name='dc', amplitude=2.5, duration=10, t_start=0)
        self.assertNotEqual(pulse1, pulse3)
    
    def test_add_remove_pulse(self):
        if self.pulse_sequence:
            isempty = False
        else:
            isempty = True
        self.assertTrue(isempty)
        
        pulse = DCPulse(name='dc', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add(pulse)
        self.assertIn(pulse, self.pulse_sequence)
        
        if self.pulse_sequence:
            isempty = False
        else:
            isempty = True
        self.assertFalse(isempty)
        
        # Remove pulses
        self.pulse_sequence.clear()
        self.assertEqual(len(self.pulse_sequence.pulses), 0)

        if self.pulse_sequence:
            isempty = False
        else:
            isempty = True
        self.assertTrue(isempty)

    def test_sort(self):
        pulse1 = DCPulse(name='dc1', amplitude=1.5, duration=10, t_start=1)
        pulse2 = DCPulse(name='dc2', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add([pulse1, pulse2])
        self.assertEqual(pulse2, self.pulse_sequence[0])

    def test_get_pulses(self):
        self.assertListEqual(self.pulse_sequence.get_pulses(), [])
        pulse1 = DCPulse(name='dc1', amplitude=1.5, duration=10, t_start=1)
        pulse2 = DCPulse(name='dc2', amplitude=2.5, duration=10, t_start=1)
        pulse3 = TriggerPulse(name='trig', duration=12, t_start=1)
        self.pulse_sequence.add([pulse1, pulse2, pulse3])
        
        subset_pulses = self.pulse_sequence.get_pulses()
        self.assertListEqual(subset_pulses, [pulse1, pulse2, pulse3])
        subset_pulses = self.pulse_sequence.get_pulses(t_start=1)
        self.assertListEqual(subset_pulses, [pulse1, pulse2, pulse3])
        subset_pulses = self.pulse_sequence.get_pulses(duration=10)
        self.assertListEqual(subset_pulses, [pulse1, pulse2])
        subset_pulses = self.pulse_sequence.get_pulses(amplitude=1.5)
        self.assertListEqual(subset_pulses, [pulse1])
        subset_pulses = self.pulse_sequence.get_pulses(amplitude=('>',1.5))
        self.assertListEqual(subset_pulses, [pulse2])
        subset_pulses = self.pulse_sequence.get_pulses(amplitude=('>=',1.5))
        self.assertListEqual(subset_pulses, [pulse1, pulse2])
        
        pulse = self.pulse_sequence.get_pulse(amplitude=1.5)
        self.assertEqual(pulse, pulse1)
        pulse = self.pulse_sequence.get_pulse(duration=12)
        self.assertEqual(pulse, pulse3)
        with self.assertRaises(RuntimeError):
            pulse = self.pulse_sequence.get_pulse(duration=10)

    def test_transition_voltages(self):
        # To test transitions, pulses must be on the same connection
        channel_out = Channel('arbstudio', 'ch1', id=1, output=True)
        channel_in = Channel('device', 'input', id=1, output=True)
        c1 = SingleConnection(output_instrument='arbstudio', output_channel=channel_out,
                              input_instrument='device', input_channel=channel_in)
        pulses = [DCPulse(name='dc1', amplitude=0, duration=5, t_start=0, connection=c1),
                  DCPulse(name='dc2', amplitude=1, duration=10, t_start=5, connection=c1),
                  DCPulse(name='dc3', amplitude=2, duration=8, t_start=15, connection=c1),
                  DCPulse(name='dc4', amplitude=3, duration=7, t_start=12, connection=c1)]
        
        self.pulse_sequence.add(pulses)
        
        self.assertRaises(TypeError, self.pulse_sequence.get_transition_voltages)
        self.assertRaises(TypeError, self.pulse_sequence.get_transition_voltages, connection=c1)
        self.assertRaises(TypeError, self.pulse_sequence.get_transition_voltages, t=5)
        
        transition_voltage = self.pulse_sequence.get_transition_voltages(pulse=pulses[1])
        self.assertTupleEqual(transition_voltage, (0, 1))
        
        transition_voltage = self.pulse_sequence.get_transition_voltages(connection=c1, t=5)
        self.assertTupleEqual(transition_voltage, (0, 1))
        
        transition_voltage = self.pulse_sequence.get_transition_voltages(connection=c1, t=15)
        self.assertTupleEqual(transition_voltage, (1, 2))

In [4]:
class TestArbStudio(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()
        self.arbstudio = MockArbStudio(name='mock_arbstudio', server_name='')
        self.arbstudio_interface = get_instrument_interface(self.arbstudio)
        
    def tearDown(self):
        self.arbstudio.close()
        self.arbstudio_interface.close()
    
    def test_pulse_implementation(self):
        sine_pulse = SinePulse(t_start=0, duration=10, frequency=1e6, amplitude=1)
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(sine_pulse))
        
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        self.assertIsNotNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        DC_pulse.amplitude=3
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        
    def test_ELR_programming(self):
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            self.arbstudio_interface.pulse_sequence(('add', pulse))
        self.assertEqual(pulses, self.arbstudio_interface.pulse_sequence().pulses)

In [5]:
class TestATS(unittest.TestCase):
    def setUp(self):
        silent = True
        self.ATS = MockATS(name='ATS', server_name='ATS_server', silent=silent)
        self.acquisition_controller = MockAcquisitionController(name='basic_acquisition_controller',
                                                                alazar_name='ATS',
                                                                server_name='ATS_server',
                                                                silent=silent)

    def tearDown(self):
        self.ATS.close()
        self.acquisition_controller.close()
        self.ATS_interface.close()

    def test_initialize_with_acquisition_controller(self):
        from silq.instrument_interfaces.AlazarTech.ATS_interface import ATSInterface
        self.ATS_interface = ATSInterface(instrument_name='ATS', 
                                          acquisition_controller_names=['basic_acquisition_controller'],
                                          server_name='ATS_server')

    def test_add_acquisition_controller(self):
        self.ATS_interface = get_instrument_interface(self.ATS)
        self.ATS_interface.add_acquisition_controller('basic_acquisition_controller')

    def test_interface_settings(self):
        self.ATS_interface = get_instrument_interface(self.ATS)
        self.ATS_interface.add_acquisition_controller('basic_acquisition_controller')
        
        with self.assertRaises(KeyError):
            self.ATS_interface.setting('trigger_slope1')
        
        self.ATS_interface.set_configuration_settings(trigger_slope1='positive')
        self.assertEqual(self.ATS_interface.setting('trigger_slope1'), 'positive')
        
        self.ATS_interface.update_settings(trigger_slope2='positive')
        self.assertEqual(self.ATS_interface.setting('trigger_slope1'), 'positive')
        self.ATS_interface.update_settings(trigger_slope1='negative')
        self.assertEqual(self.ATS_interface.setting('trigger_slope1'), 'negative')
        self.assertEqual(self.ATS_interface.setting('trigger_slope2'), 'positive')

    def test_setup_ATS(self):
        self.ATS_interface = get_instrument_interface(self.ATS)
        self.ATS_interface.add_acquisition_controller('basic_acquisition_controller')
        
        self.ATS_interface.update_settings(trigger_engine1='J',
                                           trigger_source1='positive',
                                           records_per_buffer=True)
        self.assertNotIn('trigger_engine1', self.ATS.configuration_settings())
        self.assertNotIn('records_per_buffer', self.ATS.configuration_settings())
        self.ATS_interface.setup_ATS()
        self.assertIn('trigger_engine1', self.ATS.configuration_settings())
        self.assertNotIn('records_per_buffer', self.ATS.configuration_settings())

    def test_setup_acquisition_controller(self):
        self.ATS_interface = get_instrument_interface(self.ATS)
        self.ATS_interface.add_acquisition_controller('basic_acquisition_controller',
                                                      cls_name='Basic_AcquisitionController')
        
        self.ATS_interface.average_mode('point')
        self.ATS_interface.setup()
        
        self.ATS_interface.update_settings(trigger_engine1='J',
                                           trigger_source1='positive',
                                           records_per_buffer=True)
        self.assertNotIn('trigger_engine1', self.ATS.configuration_settings())
        self.assertNotIn('records_per_buffer', self.ATS.configuration_settings())
        self.assertEqual(self.acquisition_controller.average_mode(), 'point')
        
        self.ATS_interface.setup_acquisition_controller()
        self.assertNotIn('trigger_engine1', self.acquisition_controller.acquisition_settings())
        self.assertIn('records_per_buffer', self.acquisition_controller.acquisition_settings())
        
        

In [9]:
class TestLayout(unittest.TestCase):
    def setUp(self):
        silent = True
        self.arbstudio = MockArbStudio(name='arbstudio', silent=silent, server_name='')
        self.pulseblaster = MockPulseBlaster(name='pulseblaster', silent=silent, server_name='')
        self.chip = Chip(name='chip', server_name='')
        
        self.ATS = MockATS(name='ATS', server_name='ATS_server', silent=silent)
        self.acquisition_controller = MockAcquisitionController(name='basic_acquisition_controller',
                                                                alazar_name='ATS',
                                                                server_name='ATS_server',
                                                                silent=silent)
        
        self.instruments = [self.arbstudio, self.pulseblaster, self.chip, self.ATS]
        self.interfaces = {instrument.name: get_instrument_interface(instrument) 
                                      for instrument in self.instruments}
        
        self.interfaces['ATS'].add_acquisition_controller('basic_acquisition_controller',
                                                                     cls_name='Basic_AcquisitionController')
        self.layout = Layout(name='layout',
                             instrument_interfaces = list(self.interfaces.values()),
                             server_name='layout_server')
        
        self.instruments += [self.acquisition_controller, self.layout]

    def tearDown(self):
        for instrument in self.instruments:
            instrument.close()
        for instrument_interface in self.interfaces.values():
            instrument_interface.close()

    def test_connections(self):
        self.layout.add_connection(output_arg='arbstudio.ch1',
                                   input_arg='chip.TGAC', default=True)
        self.layout.add_connection(output_arg='arbstudio.ch2',
                                   input_arg='chip.DF')
        self.layout.primary_instrument('arbstudio')
        connections = self.layout.get_connections()
        self.assertEqual(len(connections), 2)
        
        connections = self.layout.get_connections(input_channel='DF')
        self.assertEqual(len(connections), 1)
        self.assertEqual(connections[0].input['channel'].name, 'DF')

        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        connection = self.layout.get_pulse_connection(DC_pulse)
        self.assertEqual(connection.output['channel'].name, 'ch1')

    def test_pulse_implementation(self):
        self.layout.add_connection(output_arg='arbstudio.ch1',
                                   input_arg='chip.TGAC', default=True)
        self.layout.add_connection(output_arg='arbstudio.ch2',
                                   input_arg='chip.DF')
        self.layout.primary_instrument('arbstudio')
        
        sine_pulse = SinePulse(t_start=0, duration=10, frequency=1e6, amplitude=1)
        with self.assertRaises(Exception):
            pulse_instrument = self.layout.get_pulse_instrument(sine_pulse)
            
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        pulse_instrument = self.layout.get_pulse_instrument(DC_pulse)
        self.assertEqual(pulse_instrument, 'arbstudio')
        
        DC_pulse2 = DCPulse(t_start=0, duration=10, amplitude=4)
        with self.assertRaises(Exception):
            pulse_instrument = self.layout.get_pulse_instrument(DC_pulse2)

    def test_setup_pulses(self):
        self.layout.primary_instrument('arbstudio')
        self.layout.add_connection(output_arg='arbstudio.ch1',
                                   input_arg='chip.TGAC')
        self.layout.add_connection(output_arg='arbstudio.ch2',
                                   input_arg='chip.DF', default=True)
        
        pulse_sequence = PulseSequence()
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            pulse_sequence.add(pulse)
            
        self.layout.target_pulse_sequence(pulse_sequence)
        self.layout.setup()
        
        waveforms = self.arbstudio.get_waveforms()
        for channel in [0,2,3]:
            self.assertEqual(len(waveforms[channel]), 0)
        self.assertEqual(len(waveforms[1]), 3)
        sequence = [self.arbstudio.ch1_sequence(),
                    self.arbstudio.ch2_sequence(),
                    self.arbstudio.ch3_sequence(),
                    self.arbstudio.ch4_sequence()]
        for channel in [0,2,3]:
            self.assertEqual(len(sequence[channel]), 0)
        self.assertEqual(len(sequence[1]), 3)

    def test_arbstudio_pulseblaster(self):
        self.layout.primary_instrument('pulseblaster')
        self.layout.add_connection(output_arg='arbstudio.ch1',
                                   input_arg='chip.TGAC')
        self.layout.add_connection(output_arg='arbstudio.ch2',
                                   input_arg='chip.DF', default=True)
        
        self.layout.add_connection(output_arg='pulseblaster.ch1',
                                   input_arg='arbstudio.trig_in',
                                   trigger=True)
        
        trigger_connections = self.layout.get_connections(
            input_instrument='arbstudio', trigger=True)
        self.assertEqual(len(trigger_connections), 1)
        trigger_connection = trigger_connections[0]
        self.assertEqual(trigger_connection.output['instrument'], 'pulseblaster')

        pulse_sequence = PulseSequence()
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            pulse_sequence.add(pulse)
            
        self.layout.target_pulse_sequence(pulse_sequence)
        self.layout.setup()
        
        self.assertEqual(len(self.pulseblaster.instructions()), 7)
        self.assertEqual([ins[0] for ins in self.pulseblaster.instructions()], [1,0,1,0,1,0,0])
        self.assertEqual(self.pulseblaster.instructions()[-1][2], 1)
        
        self.pulseblaster.instructions([])
        self.interfaces['pulseblaster'].ignore_first_trigger(True)
        self.layout.target_pulse_sequence(pulse_sequence)
        self.layout.setup()
        
        self.assertEqual(len(self.pulseblaster.instructions()), 7)
        self.assertEqual([ins[0] for ins in self.pulseblaster.instructions()], [0,0,1,0,1,0,1])

    def test_full_setup(self):
        # Setup connectivity to match the Berdina setup
        self.layout.primary_instrument('pulseblaster')
        self.layout.acquisition_instrument('ATS')
        self.layout.add_connection(output_arg='pulseblaster.ch1',
                                   input_arg='arbstudio.trig_in',
                                   trigger=True)
        self.layout.add_connection(output_arg='pulseblaster.ch2',
                                   input_arg='ATS.trig_in',
                                   trigger=True)
        
        self.layout.add_connection(output_arg='arbstudio.ch1',
                                   input_arg='chip.TGAC')
        self.layout.add_connection(output_arg='arbstudio.ch2',
                                   input_arg='chip.DF', default=True)
        
        self.layout.add_connection(output_arg='chip.output',
                                   input_arg='ATS.chA')
        self.interfaces['ATS'].acquisition_channels(['chC', 'chA'])
        
        # Setup pulses
        pulse_sequence = PulseSequence()
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0, acquire=True)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            pulse_sequence.add(pulse)
            
        self.layout.target_pulse_sequence(pulse_sequence)
        self.layout.setup()
        
        # Test Pulseblaster
        # print('pulseblaster pulsesequence: {}'.format(self.interfaces['pulseblaster'].pulse_sequence()))
        # print('Pulseblaster instructions:\n{}\n\n'.format(self.pulseblaster.instructions()))
        self.assertEqual(len(self.pulseblaster.instructions()), 7)
        self.assertEqual([ins[0] for ins in self.pulseblaster.instructions()], [1,0,1,0,3,0,0])
        self.assertEqual(self.pulseblaster.instructions()[-1][2], 1)
        
        # Test ATS
        ATS_pulse_sequence = self.interfaces['ATS'].pulse_sequence()
        self.assertEqual(self.interfaces['ATS'].active_acquisition_controller(),
                        'basic_acquisition_controller')
        self.assertEqual(self.interfaces['ATS'].trigger_slope(), 'positive')
        self.assertTrue(0 < self.interfaces['ATS'].trigger_threshold() < 3.3)
        self.assertEqual(len(ATS_pulse_sequence), 2)
        configuration_settings = self.ATS.configuration_settings()
        acquisition_settings = self.acquisition_controller.acquisition_settings()
        self.assertEqual(configuration_settings['trigger_engine1'], 'J')
        self.assertEqual(len(acquisition_settings), 1)
        self.assertEqual(acquisition_settings['channel_selection'], 'AC')
#         print('acquisition_controller settings: {}'.format(
#                 self.acquisition_controller.acquisition_settings()))
#         print('ATS settings: {}'.format(
#                 self.ATS.configuration_settings()))

In [10]:
# Run tests    
suite_pulse_sequence = unittest.TestLoader().loadTestsFromModule(TestPulseSequence())
suite_arbstudio = unittest.TestLoader().loadTestsFromModule(TestArbStudio())
suite_ATS = unittest.TestLoader().loadTestsFromModule(TestATS())
suite_layout = unittest.TestLoader().loadTestsFromModule(TestLayout())
suite = unittest.TestSuite([suite_pulse_sequence, suite_arbstudio, suite_ATS, suite_layout])
# suite = unittest.TestSuite([suite_layout])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_add_remove_pulse (__main__.TestPulseSequence) ... ok
test_get_pulses (__main__.TestPulseSequence) ... ok
test_pulse_equality (__main__.TestPulseSequence) ... ok
test_sort (__main__.TestPulseSequence) ... ok
test_transition_voltages (__main__.TestPulseSequence) ... ok
test_ELR_programming (__main__.TestArbStudio) ... ok
test_pulse_implementation (__main__.TestArbStudio) ... ok
test_add_acquisition_controller (__main__.TestATS) ... ok
test_initialize_with_acquisition_controller (__main__.TestATS) ... ok
test_interface_settings (__main__.TestATS) ... ok
test_setup_ATS (__main__.TestATS) ... ok
test_setup_acquisition_controller (__main__.TestATS) ... ok
test_arbstudio_pulseblaster (__main__.TestLayout) ... ok
test_connections (__main__.TestLayout) ... ok
test_full_setup (__main__.TestLayout) ... ok
test_pulse_implementation (__main__.TestLayout) ... ok
test_setup_pulses (__main__.TestLayout) ... ok

----------------------------------------------------------------------
Ran 17 tests in

<unittest.runner.TextTestResult run=17 errors=0 failures=0>

In [8]:
class A:
    def __init__(self):
        self.x = 0
        
    def __bool__(self):
        return self.x == 1
        
    def __len__(self):
        return 3
    
    @property
    def get_val(self):
        return 5
    
class B(A):
    def __init__(self):
        self.x = 0

a = A()
if a:
    print('intially True')
a.x = 1
if a:
    print('True now x=1')
print('length of x: {}'.format(len(a)))

b = B()
print('b is subclass of a: {}'.format(isinstance(b, A)))

True now x=1
length of x: 3
b is subclass of a: True
